In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import unidecode

def normalize(text):
    return unidecode.unidecode(text).upper()

df_embalses = pd.read_csv('embalsesUTF8.csv')  # Archivo con ID, AMBITO_NOMBRE, EMBALSE_NOMBRE, etc.
df_listado = pd.read_csv('listadoUTF8-3.tsv', delimiter='\t')  # Archivo con CODIGO, NOMBRE, EMBALSE, etc.

df_listado['EMBALSE_NORMALIZADO'] = df_listado['EMBALSE'].apply(normalize)

embalse_list = df_listado['EMBALSE_NORMALIZADO'].tolist()

merge_result = []
similarity_threshold = 75

for index_embalses, row_embalses in df_embalses.iterrows():
    embalse_nombre = normalize(row_embalses['EMBALSE_NOMBRE'])
    
    best_match, best_score = process.extractOne(
        embalse_nombre, 
        embalse_list,
        scorer=fuzz.token_set_ratio
    )
    
    if best_score > similarity_threshold:
        best_match_row = df_listado[df_listado['EMBALSE_NORMALIZADO'] == best_match].iloc[0]
        combined_row = {**row_embalses, **best_match_row}
        merge_result.append(combined_row)
        print("match:", embalse_nombre, "<->", best_match, "score:", best_score)

df_combinado = pd.DataFrame(merge_result)
df_combinado.to_csv('embalses_combinado.csv', index=False)